In [2]:
import torch
import torch.nn as nn
import clip
import matplotlib.pyplot as plt

clip,_=clip.load("ViT-B/32",device="cuda")
token_layer =clip.token_embedding
token_weights= token_layer.weight
#print(token_weights.shape) #49408,512

normed_weights=token_weights/torch.norm(token_weights,dim=1).unsqueeze(1)
token_similarity=torch.matmul(normed_weights,normed_weights.T).detach().cpu().numpy()

#plot the similarity matrix

plt.imshow(token_similarity)
plt.colorbar()
plt.show()
#we can see that the similarity matrix is very sparse, which means that the tokens are not very similar to each other. This is a good thing, as it means that the tokens are not redundant.

OutOfMemoryError: CUDA out of memory. Tried to allocate 9.10 GiB (GPU 0; 15.89 GiB total capacity; 451.13 MiB already allocated; 1.37 GiB free; 9.55 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

The language model were using for translation is going to go straight  into a CLIP model. 

For human readability it's natural to get as far as a tokenized output before applying the CLIP tokenization layers.

To do this the long way, we use the equivalence of a onehot@token.weight === embedding(token_idx) to preserve the gradient over the transition. Unfortunately, it's very intensive on VRAM! 

There must be a better way! 


Here's where we enquire of the penultimate layers to the LM to see if the language head has any useful features. 


```

class myMarianMTModel(MarianMTModel):
     def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        decoder_input_ids: Optional[torch.LongTensor] = None,
        decoder_attention_mask: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        decoder_head_mask: Optional[torch.Tensor] = None,
        cross_attn_head_mask: Optional[torch.Tensor] = None,
        encoder_outputs: Optional[Union[Tuple[torch.Tensor], BaseModelOutput]] = None,
        past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Seq2SeqLMOutput:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the masked language modeling loss. Indices should either be in `[0, ...,
            config.vocab_size]` or -100 (see `input_ids` docstring). Tokens with indices set to `-100` are ignored
            (masked), the loss is only computed for the tokens with labels in `[0, ..., config.vocab_size]`.

        Returns:

        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        #labels is none so we can ignore...
        # if labels is not None:
        #     if use_cache:
        #         logger.warning("The `use_cache` argument is changed to `False` since `labels` is provided.")
        #     use_cache = False
        #     if decoder_input_ids is None and decoder_inputs_embeds is None:
        #         decoder_input_ids = shift_tokens_right(
        #             labels, self.config.pad_token_id, self.config.decoder_start_token_id
        #         )

        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            encoder_outputs=encoder_outputs,
            decoder_attention_mask=decoder_attention_mask,
            head_mask=head_mask,
            decoder_head_mask=decoder_head_mask,
            cross_attn_head_mask=cross_attn_head_mask,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            decoder_inputs_embeds=decoder_inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        lm_logits = self.lm_head(outputs[0]) + self.final_logits_bias
        EOT=
        return Seq2SeqLMOutput(
            model_outputs=outputs[0],
            logits=lm_logits,
            past_key_values=outputs.past_key_values,
            decoder_hidden_states=outputs.decoder_hidden_states,
            decoder_attentions=outputs.decoder_attentions,
            cross_attentions=outputs.cross_attentions,
            encoder_last_hidden_state=outputs.encoder_last_hidden_state,
            encoder_hidden_states=outputs.encoder_hidden_states,
            encoder_attentions=outputs.encoder_attentions,
        )
```
